### cnn网络结构
借鉴了《matlab deeplearning》第六章的网络结构

![cnn网络结构](images/2023-05-14-16-42-53.png)

具体的网络维数

![cnn网络结构](images/2023-05-14-16-46-02.png)

具体的反向传播梯度计算过程过程见邱锡朋《nndl》

### 分布式admm算法：
$$ \min_{W}\sum_{i=1}^{n} L_{i}(x_{i},y_{i};W) $$
等价于
$$ \min_{W_{1} \cdots W_{n},Z} \sum_{i=1}^{n} L_{i}(x_{i},y_{i};W_{i}) $$
$$ s.t. \quad W_{i} = Z \quad (i = 1,\cdots , n) $$
拉格朗日函数为：
$$ L_{c} = \sum_{i=1}^{n}L_{i}(x_{i},y_{i};W_{i}) + \sum_{i=1}^{n} V_{i}\odot 
(W_{i}-Z_{i}) + \frac{c}{2} \sum_{i=1}^{n}\left \| W_{i} -Z \right\|_{F}^{2} $$
迭代公式
$$1.\quad W_{i}^{k+1} = arg\min_{W_{i}} (L_{i}(x_{i},y_{i};W_{i}) + V_{i}^{k}\odot (W_{i}-Z_{i}^{k}) + \frac{c}{2}\left \| W_{i} - Z^{k} \right \|_{F}^{2})$$
$$\quad\qquad= arg\min_{W_{i}} (L_{i}(x_{i},y_{i};W_{i}) +  \frac{c}{2}\left \| W_{i} - Z^{k}  + \frac{V_{i}^{k}}{c} \right \|_{F}^{2})\quad (\forall i) $$
$$2.\quad  Z^{k+1} = arg \min_{Z}\frac{c}{2} \sum_{i=1}^{n} \left \|Z -  W_{i}^{k+1} - \frac{V_{i}^{k}}{c} \right\|_{F}^{2} $$
$$= \frac{1}{n}\sum_{i=1}^{n}( W_{i}^{k+1} + \frac{V_{i}^{k}}{c}) $$
$$3.V_{i}^{k+1} = V_{i}^{k} + c(W_{i}^{k} - Z^{k+1}) \qquad (\forall i)$$

### admm算法与nn相结合就有：

1. 其中迭代公式$1.$中$W^{k+1}_{i}$的更新需要用梯度下降的方式
2. 其实这个分布式的算法的可以优化的地方也只有$W^{k+1}_{i}$的更新过程了，这个更新的过程根本就没有什么精确解，只能用神经网络训练的方式去求近视解$W^{k+1}_{i}$
3. 这其实是有一个主机Z的，Z瘫痪了就寄了，可以去主机吗？
4. to be continue...

### 问题及解答：

1. 在cnn_admm_first \ test_cnn_admm.ipynb中admm算法实现是有问题的，每次迭代过程$L_{i}(x_{i},W_{i})$由于输入$x_{i}$的变化改变了，相当于admm同步的过程只是走了一步，相当于是实现了“一步”admm算法，其实也是work的，但是不是最原滋原味的admm。
2. 上面的理解其实是错误的，
3. Q:问题中的$$ \min_{W}\sum_{i=1}^{n} L_{i}(x_{i},y_{i};W) $$中的$$L_{i}(x_{i},y_{i};W)$$是什么意思？A:其实也就是在agent_i节点的所有输入和输出的损失的和。应该改成$$L_{i}(X_{i},Y_{i};W)$$其中$X_{i},Y_{i}$是在agent_i所有的输入和输出，但是也可以不改，记住这个区别就行了。
4. Q:每个agent_i怎么求$$L_{i}(X_{i},Y_{i};W_{i})$$极小点$W_{i}$的？A:其实就是多次的梯度下降过程，直到下降到局部的最小点处，但是为了后来避免了全部数据进行训练的缺点，后来就使用每次用batch数量的数据进行梯度下降。
5. Q:在nn\nn.py和cnn_admm_first\conv_nn.py写的神经网络的训练过程的train函数有什么问题？A:我每一次训练的时候只是喂进去的只有一个数据就进行梯度下降了，相当于batch_size=1，所以说是可能出现问题的，batch_size应该可调好一些。

### 各文件的作用：

1. nn是全连接网络及其与admm结合实现分布式
2. cnn_admm_first是cnn和admm结合实现的分布式的第一次尝试，有点问题

### 引用
